In [1]:
! gdown 1yEjicXasMGFaALQobQWgsSxQgLEtxruC
! unzip /content/ThaiDepression.zip

Downloading...
From: https://drive.google.com/uc?id=1yEjicXasMGFaALQobQWgsSxQgLEtxruC
To: /content/ThaiDepression.zip
100% 41.1k/41.1k [00:00<00:00, 32.7MB/s]
Archive:  /content/ThaiDepression.zip
  inflating: ThaiDepression/AHB_01.csv  
  inflating: ThaiDepression/AHB_02.csv  
  inflating: ThaiDepression/AHB_03.csv  
  inflating: ThaiDepression/AHB_04.csv  
  inflating: ThaiDepression/AHB_05.csv  
  inflating: ThaiDepression/AHB_06.csv  
  inflating: ThaiDepression/AHB_07.csv  
  inflating: ThaiDepression/AHB_08.csv  
  inflating: ThaiDepression/AHB_09.csv  
  inflating: ThaiDepression/AHB_10.csv  
  inflating: ThaiDepression/AHB_11.csv  
  inflating: ThaiDepression/AHB_12.csv  
  inflating: ThaiDepression/AHB_13.csv  
  inflating: ThaiDepression/All_AHB.csv  


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11348249646624951180
xla_global_id: -1
]


In [3]:
import pandas as pd
import numpy as np
import os
from numpy import array
from math import sqrt
from numpy import mean
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from keras.preprocessing import sequence
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, GRU, BatchNormalization
from pandas import read_csv
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, SimpleRNN
import json
import psutil 
from datetime import datetime, timezone, timedelta

In [4]:
file_list = os.listdir('ThaiDepression')
file_list.sort()
df_list = []
for i in range(14):
    df_list.append(pd.read_csv('ThaiDepression/' + file_list[i], index_col=0))

In [5]:
df_list

[         Chiang_Rai    Nan  Phayao  Phrae  Chiang_Mai  Mae_Hong_Son  Lampang  \
 2016-10       10383   5315    4266   4057       15408          2002     9011   
 2016-11       10383   5315    4265   4057       15406          2002     9011   
 2016-12       10382   5315    4265   4057       15404          2002     9011   
 2017-01       10382   5655    4265   4057       15393          2002     9011   
 2017-02       10382   5655    4264   4057       15392          2002     9011   
 ...             ...    ...     ...    ...         ...           ...      ...   
 2022-03       25223   9711   10788   7988       47292          6924    17444   
 2022-04       25456   9781   10932   8095       48020          7004    17606   
 2022-05       25619   9833   10977   8122       48155          7026    17703   
 2022-06       25836   9884   11069   8161       48616          7090    17783   
 2022-07       26174  10019   11163   8250       49096          7169    17951   
 
          Lamphun   AHB_1 

In [6]:
# split a univariate sequence into samples
def uni_split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


    # split a multivariate sequence into samples
def multi_split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


def get_scaler(scaler):
    scalers = {
        "minmax": MinMaxScaler,
        "standard": StandardScaler,
        "maxabs": MaxAbsScaler,
        "robust": RobustScaler,
    }
    return scalers.get(scaler.lower())()

In [ ]:
%matplotlib inline
# define scope of configs
AHB_list = [6, 7]
BiLSTM_hidden_units = [8, 16, 32, 64, 128]
n_layer = [1, 2, 3]
n_batch = [2, 4, 8, 16, 32, 64, 128]
n_steps = [2]
patience_list = [3, 5, 7, 10]
mul_uni = ['uni','mul']
dropouts = [0.2, 0.3, 0.5]


timezone_offset = 7.0  # Bangkok Time (UTC+07:00)
tzinfo = timezone(timedelta(hours=timezone_offset))
count = 1
performance_json = []
for g in AHB_list:
    for h in patience_list:
        for a in n_steps:
            for b in dropouts:
                for c in BiLSTM_hidden_units:
                    for d in n_layer:
                        for e in n_batch:
                            for f in mul_uni:

                                print('PROGRESS: [ ' + str(count) + ' / ' + str(len(dropouts)*len(AHB_list)*len(n_steps)*len(BiLSTM_hidden_units)*len(n_layer)*len(n_batch)*len(patience_list)*len(mul_uni)) + ' ]')
                                
                                if f == 'uni':
                                    if g == 13:
                                        raw_seq = list(df_list[g]['Country_Level'])
                                        ahb = 'Country_Level'
                                    else:    
                                        raw_seq = list(df_list[g]['AHB_'+str(g+1)])
                                        ahb = 'AHB_'+str(g+1)
                                    n_features = 1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq).reshape(len(raw_seq),-1))
                                    X, y = uni_split_sequence(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
                                if f == 'mul':
                                    if g == 13:
                                        raw_seq = list(df_list[13].drop(df_list[13].columns[:77], axis=1).values)
                                        ahb = 'Country_Level'
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[13]['Country_Level']).reshape(len(df_list[13]['Country_Level']),-1))
                                    else:    
                                        raw_seq = list(df_list[g].drop(['Country_Level'], axis=1).values)
                                        ahb = 'AHB_'+str(g+1)
                                        scaler_y = get_scaler('minmax')
                                        scaler_y.fit(np.array(df_list[g]['AHB_'+str(g+1)]).reshape(len(df_list[13]['AHB_'+str(g+1)]),-1))
                                    n_features = len(raw_seq[0])-1
                                    scaler = get_scaler('minmax')
                                    raw_seq_arr = scaler.fit_transform(np.array(raw_seq))
                                    X, y = multi_split_sequences(raw_seq_arr, a)
                                    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, train_size=0.9, shuffle=False, stratify=None)
                                    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
                                    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))


                                callback = EarlyStopping(monitor='loss', patience=h)
                                model = Sequential()
                                if d == 1:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                if d == 2:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, activation='tanh')))
                                    model.add(Dropout(b))
                                if d == 3:
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=True, input_shape=(a,n_features), activation='tanh')))
                                    model.add(Dropout(b))
                                    model.add(Bidirectional(LSTM(units=c, return_sequences=False, activation='tanh')))
                                    model.add(Dropout(b))
                                model.add(Dense(units=1, kernel_initializer='he_uniform', activation='linear'))
                                model.compile(loss='mse', optimizer='adam')  
                                history = model.fit(X_train, y_train, epochs=2000, batch_size=e, verbose=0, callbacks=[callback])
                                last_epoch = len(history.history['loss']) 

                                yhat = model.predict(X_test, verbose=0)
                                yhat = yhat.reshape(yhat.shape[0],-1)
                                mae = mean_absolute_error(y_test, yhat)
                                mse = mean_squared_error(y_test, yhat)
                                rmse = mean_squared_error(y_test, yhat, squared = False)
                                mape = mean_absolute_percentage_error(y_test, yhat)
                                
                                if f == 'uni':
                                    yhat_inverse = scaler.inverse_transform(yhat)
                                    y_test_inverse = scaler.inverse_transform(y_test)
                                if f == 'mul':
                                    yhat_inverse = scaler_y.inverse_transform(yhat)
                                    y_test_inverse = scaler_y.inverse_transform(y_test.reshape(len(y_test),-1))


                                mae_inverse = mean_absolute_error(y_test_inverse, yhat_inverse)
                                mse_inverse = mean_squared_error(y_test_inverse, yhat_inverse)
                                rmse_inverse = mean_squared_error(y_test_inverse, yhat_inverse, squared = False)
                                mape_inverse = mean_absolute_percentage_error(y_test_inverse, yhat_inverse)

                                performance_json.append({'model': 'BiLSTM', 'AHB_number': ahb, 'Multi_Uni': f, \
                                                         'n_layer': d, 'BiLSTM_hidden_units': c, 'dropouts': b,'n_batch': e,\
                                                         'n_steps': a, 'last_epoch': last_epoch, 'patience_list': h, 'MAE_normalize': float(mae), \
                                                         'MSE_normalize': float(mse), 'RMSE_normalize': float(rmse), 'MAPE_normalize': float(mape), \
                                                         'MAE': float(mae_inverse), 'MSE': float(mse_inverse), 'RMSE': float(rmse_inverse), 'MAPE': float(mape_inverse)})
                                

                                json_formatted_str = json.dumps(performance_json[-1], indent=2)
                                print(json_formatted_str)
                                print('Percent CPU Usage: ' + str(psutil.cpu_percent()))
                                print('Percent Ram Usage: ' + str(psutil.virtual_memory()[2]))
                                print(datetime.now(tzinfo))

                                count = count + 1

PROGRESS: [ 1 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_7",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 2,
  "last_epoch": 20,
  "patience_list": 3,
  "MAE_normalize": 0.1269855370637494,
  "MSE_normalize": 0.016179131977611432,
  "RMSE_normalize": 0.12719721686267915,
  "MAPE_normalize": 0.1316971634400816,
  "MAE": 6039.940848214282,
  "MSE": 36602612.29008261,
  "RMSE": 6050.009280164999,
  "MAPE": 0.060210251053874696
}
Percent CPU Usage: 5.0
Percent Ram Usage: 11.4
2022-10-06 11:37:58.537217+07:00
PROGRESS: [ 2 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_7",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 2,
  "n_steps": 2,
  "last_epoch": 12,
  "patience_list": 3,
  "MAE_normalize": 0.08309559727874574,
  "MSE_normalize": 0.006944544726888108,
  "RMSE_normalize": 0.08333393502582312,
  "MAPE_normalize": 0.08614131319533129,
  "MAE": 3952.3560

{
  "model": "BiLSTM",
  "AHB_number": "AHB_7",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 2,
  "last_epoch": 18,
  "patience_list": 3,
  "MAE_normalize": 0.20820000758649373,
  "MSE_normalize": 0.04350538545474684,
  "RMSE_normalize": 0.20857944638613565,
  "MAPE_normalize": 0.2158945411736608,
  "MAE": 9902.82366071428,
  "MSE": 98423687.7232491,
  "RMSE": 9920.871318752657,
  "MAPE": 0.09871177762641403
}
Percent CPU Usage: 54.0
Percent Ram Usage: 12.2
2022-10-06 11:38:22.997780+07:00
PROGRESS: [ 6 / 5040 ]


{
  "model": "BiLSTM",
  "AHB_number": "AHB_7",
  "Multi_Uni": "mul",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 8,
  "n_steps": 2,
  "last_epoch": 15,
  "patience_list": 3,
  "MAE_normalize": 0.13259279293129875,
  "MSE_normalize": 0.01771472994028401,
  "RMSE_normalize": 0.13309669394948925,
  "MAPE_normalize": 0.13740259888361722,
  "MAE": 6306.642857142853,
  "MSE": 40076628.26161407,
  "RMSE": 6330.610417772845,
  "MAPE": 0.06284599543362088
}
Percent CPU Usage: 63.1
Percent Ram Usage: 12.2
2022-10-06 11:38:30.526471+07:00
PROGRESS: [ 7 / 5040 ]
{
  "model": "BiLSTM",
  "AHB_number": "AHB_7",
  "Multi_Uni": "uni",
  "n_layer": 1,
  "BiLSTM_hidden_units": 8,
  "dropouts": 0.2,
  "n_batch": 16,
  "n_steps": 2,
  "last_epoch": 18,
  "patience_list": 3,
  "MAE_normalize": 0.16874670547554924,
  "MSE_normalize": 0.028584742776323325,
  "RMSE_normalize": 0.16907023030777277,
  "MAPE_normalize": 0.1749792872587494,
  "MAE": 8026.268973214282,
  "MSE": 646

In [ ]:
name = "all_performance_BiLSTM_2nsteps_AHB7-8"
json.dump(performance_json, open(name + ".json","w"))
df_json = pd.read_json('/content/' + name + '.json')
df_json.to_csv('/content/' + name + '.csv', index=False)
#! cp /content/all_performance_BiLSTM_2nsteps_AHB7-8.json /content/drive/MyDrive/Depression
! cp /content/all_performance_BiLSTM_2nsteps_AHB7-8.csv /content/drive/MyDrive/Depression

In [ ]:
df_json